# Wstęp do Machine Learning - praca wspólna

## Zbior diabetes
https://www.kaggle.com/uciml/pima-indians-diabetes-database

### Wczytanie pakietów

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing

### Eksploracja

#### Wczytanie zbioru danych

In [ ]:
data = pd.read_csv('diabetes.csv')

#### Przegląd danych w zbiorze
##### Metody
1. head
2. info
3. shape
4. describe
5. value_counts

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.value_counts()

#### Sprawdzenie udziału poszczególnych kategorii


### Wykresy
1. Pudełko
2. Histogram

In [ ]:
col_lst = data.columns.to_list()
col_lst.pop(-1)
col_lst

#### Pregnancies

In [ ]:
sns.boxplot(x='Pregnancies', data=data, color='g')
sns.displot(x=data['Pregnancies'].to_numpy(), kind='hist', bins=15, kde=True, color='g')
plt.show()

#### Glucose

In [ ]:
sns.boxplot(x='Glucose', data=data, color='#5588cc')
sns.displot(x=data['Glucose'].to_numpy(), kind='hist', bins=35, kde=True, color='#5588cc')
plt.show()

#### BloodPressure

In [ ]:
sns.boxplot(x='BloodPressure', data=data, color='r')
sns.displot(x=data['BloodPressure'].to_numpy(), kind='hist', bins=16, kde=True, color='r')
plt.show()

#### SkinThickness

In [ ]:
sns.boxplot(x='SkinThickness', data=data, color='y')
sns.displot(x=data['SkinThickness'].to_numpy(), kind='hist', bins=13, kde=True, color='y')
plt.show()

#### Insulin

In [ ]:
sns.boxplot(x='Insulin', data=data, color='gray')
sns.displot(x=data['Insulin'].to_numpy(), kind='hist', bins=17, kde=True, color='gray')
plt.show()

#### BMI

In [ ]:
sns.boxplot(x='BMI', data=data, color='turquoise')
sns.displot(x=data['BMI'].to_numpy(), kind='hist', bins=22, kde=True, color='turquoise')
plt.show()

#### DiabetesPedigreeFunction

In [ ]:
sns.boxplot(x='DiabetesPedigreeFunction', data=data, color='brown')
sns.displot(x=data['DiabetesPedigreeFunction'].to_numpy(), kind='hist', bins=15, kde=True, color='brown')
plt.show()

#### Age

In [ ]:
sns.boxplot(x='Age', data=data, color='orange')
sns.displot(x=data['Age'].to_numpy(), kind='hist', bins=15, kde=True, color='orange')
plt.show()

Po histogramach widać, że wskaźnik BMI, ciśnienia krwi oraz glukozy mają rozkład zbliżony do normalnego. 
W tych trzech kolumnach danych występują wartości zerowe, więc w dalszym etapie zostaną usunięte bądź zastąpione. 
W kolumnie SkinThickness jest bardzo dużo zer, co może sugerować brak danych tego czynnika, więc bezpieczniej będzie nie uwzględniać go w modelu.

In [ ]:
data.drop('SkinThickness', axis=1, inplace=True)

#### Sprawdzenie braków danych

In [ ]:
data.isnull().sum()

In [ ]:
# usuwanie zer w bmi, glukozie i cisnieniu

data = data[(data["Glucose"] != 0) & (data["BloodPressure"] != 0) & (data["BMI"] != 0)]

In [ ]:
data.head()

#### Korelacja

In [ ]:
correlation_matrix = data[data.columns].corr()
correlation_matrix

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
color_map = sns.diverging_palette(240, 10, n=10)
mask = np.triu(np.ones_like(correlation_matrix), k=0)
sns.heatmap(correlation_matrix,vmin=-1.,vmax=1.,cmap=color_map,mask=mask,square=True)
pass

### Feature Engineering

#### Stworzenie zmiennych 0-1 dla kategorii


In [ ]:
data.loc[(data['Age'] <= 30), "AgeCat"] = "young"
data.loc[(data['Age'] > 30) & (data['Age'] <= 50), "AgeCat"] = "middle"
data.loc[(data['Age'] > 50), "AgeCat"] = "old"

In [ ]:
data = pd.get_dummies(data, columns = ["AgeCat"], drop_first = True)

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
color_map = sns.diverging_palette(240, 10, n=10)
mask = np.triu(np.ones_like(correlation_matrix2), k=0)
sns.heatmap(correlation_matrix2,vmin=-1.,vmax=1.,cmap=color_map,mask=mask,square=True)
pass

#### Zbadanie zależności zmiennych 0-1 na target

In [ ]:
correlation_matrix2 = data[data.columns[8:11]].corr()
correlation_matrix2

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
color_map = sns.diverging_palette(240, 10, n=10)
mask = np.triu(np.ones_like(correlation_matrix2), k=0)
sns.heatmap(correlation_matrix2,vmin=-1.,vmax=1.,cmap=color_map,mask=mask,square=True)
pass

### Przygotowanie zbioru
#### Podział zbioru na treningowy i testowy

In [ ]:
columns_data = data.columns.to_list()

In [ ]:
columns_data

In [ ]:
columns_data.pop(7)

In [ ]:
columns_data.pop(6)

In [ ]:
data_x = data[columns_data]

In [ ]:
data_x

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_x, data['Outcome'], test_size = 0.2, random_state = 42)

#### Standaryzacja

In [ ]:
my_standard = StandardScaler()
my_standard.fit(X_train)

In [ ]:
X_train_sscaler = my_standard.transform(X_train)

In [ ]:
X_test_sscaler = my_standard.transform(X_test)

#### Analiza głównych składowych

In [ ]:
scaled_data = preprocessing.StandardScaler().fit_transform(data.T)

pca=PCA()
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)

In [ ]:
per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]

plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label=labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.show()

### Estymacja modelu klasyfikacyjnego

Użyj wybranego modelu klasyfikacyjnego znanego Ci z zajęć. Wyestymuj dwa modele - jeden na wystandaryzowanych danych, a drugi na PCA.  

#### Regresja logistyczna z danymi standaryzowanymi

In [ ]:
Model = LogisticRegression()
Model.fit(X_train_sscaler , y_train)
y_pred = Model.predict(X_test_sscaler)

#### Regresja logistyczna z danymi PCA

In [ ]:
data_pca = data[columns_data[0:2]]

In [ ]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(data_pca, data['Outcome'], test_size = 0.2, random_state = 42)

In [ ]:
Model_pca = LogisticRegression()
Model.fit(X_train_pca, y_train_pca)
y_pred_pca = Model.predict(X_test_pca)

### Ocena jakości modelu klasyfikacyjnego

Wyznacz accuracy, precision, lub inne miary znane Ci z zajęć, które uznasz za stosowne. Porównaj wyniki dla obydwu modeli na odpowiednich zbiorach (Dane wystandaryzowane, dane PCA)

### Wyniki - standaryzowane dane - regresja log.

In [ ]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print('accuracy is',accuracy_score(y_pred,y_test))

### Wyniki - PCA - regresja log.

In [ ]:
print(classification_report(y_test_pca, y_pred_pca))
print(confusion_matrix(y_test_pca, y_pred_pca))
print('accuracy is',accuracy_score(y_pred_pca, y_test_pca))

##### Po wynikach uzyskanych z raportu dotyczącego modeli, widać, że ten z danymi standaryzowanymi lepiej klasyfikuje przypadki cukrzycy. Jednak patrząc na wyniki w confussion matrix - jest tam ciągle dość duży odsetek pacjentów błędnie zaklasyfikowanych czy to jako zdrowi, będących de facto chorymi i na odwrót - zaklasyfikowanych jako chorzy, a będących zdrowymi. 